# Search Engine

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import os
def time_elapsed(s):
    '''
    Function to establish baselines for time needed to setup a search engine
    '''
    print("Time Elapsed: {} seconds".format(round(time.time() - s, 4)))

## Importing embedding nets

In [3]:
import pickle

In [4]:
s = time.time()
resnet152_15 = pickle.load(open("pickles/models/entire_nuswide_model_15.p", "rb"))
# resnet152_15.cpu()
# resnet152_15.eval()
time_elapsed(s)

Time Elapsed: 2.7942 seconds


## Importing datasets

`modality_data[i]` should return `(tensor, index)` of ith item

`modality_from_idx[i]` should return `value` of ith item

#### Image Dataset (NUS-WIDE)

In [5]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

In [68]:
s = time.time()

from datasets import NUS_WIDE

image_directory = 'data/Flickr'
image_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
image_data = ImageFolder(image_directory, transform = image_transform)
image_from_idx = [i[0] for i in image_data.samples]
image_data = NUS_WIDE(
    root=image_directory,
    transform=image_transform,
    feature_mode="resnet152",
    train=True)

time_elapsed(s)

Time Elapsed: 223.433 seconds


In [89]:
len(image_data.features)
len(image_from_idx)

269648

#### Text Dataset (fastText)

In [7]:
s = time.time()

fast_text = pickle.load(open("pickles/word_embeddings/word_embeddings_tensors.p", "rb"))

time_elapsed(s)

Time Elapsed: 67.3997 seconds


In [8]:
s = time.time()

text_from_idx = [None] * len(fast_text)
text_data = [None] * len(fast_text)
for idx, (key, value) in enumerate(fast_text.items()):
    text_from_idx[idx] = key
    text_data[idx] = (value, idx)

time_elapsed(s)

Time Elapsed: 0.949 seconds


## Creating Data Loaders

Need to enable batch processing of data for GPU acceleration

In [9]:
from torch.cuda import is_available

batch_size = 128
cuda = is_available()
kwargs = {'num_workers': 4, 'pin_memory': True} if cuda else {}

In [10]:
from torch.utils.data import DataLoader

image_dataloader = DataLoader(image_data, batch_size = batch_size, **kwargs)
text_dataloader = DataLoader(text_data, batch_size = batch_size, **kwargs)

## Setting up SearchEngines

In [55]:
from search import SearchEngine

In [56]:
save_directory = './embeddings'

In [57]:
search_engine = SearchEngine(["text", "image"], cuda = cuda, save_directory = save_directory, verbose = True)

In [58]:
search_engine.add_model(
        resnet152_15, "resnet152_15", 
        ["image","text"], 
        [resnet152_15.get_modOne_embedding, resnet152_15.get_modTwo_embedding],
        [(3, 224, 224), (300,)], 
        64, 
        desc = "ResNet152 trained with 15 epochs")

In [59]:
search_engine.add_dataset("fast_text", text_dataloader, text_from_idx, "text", (300,))
search_engine.add_dataset("nus-wide", image_dataloader, image_from_idx, "image", (3, 224, 224))

## Indexing data in SearchEngines

In [60]:
search_engine.build_index("fast_text", load_embeddings = False)

Building resnet152_15,fast_text index
Batch 0 of 10000
Batch 1000 of 10000
Batch 2000 of 10000
Batch 3000 of 10000
Batch 4000 of 10000
Batch 5000 of 10000
Batch 6000 of 10000
Batch 7000 of 10000
Finished building resnet152_15 index in 19.5961 seconds.


In [64]:
search_engine.build_index("nus-wide", load_embeddings = False)

Building resnet152_15,nus-wide index


RuntimeError: size mismatch, m1: [86016 x 224], m2: [2048 x 1024] at /opt/conda/conda-bld/pytorch_1532579245307/work/aten/src/THC/generic/THCTensorMathBlas.cu:249

## Queries

In [ ]:
import matplotlib.pyplot as plt

def target_to_tensor(target, modality):
    if modality == "image":
        image = PIL.Image.open(target)
        image_transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
        tensor = image_transform(image)
    elif modality == "text":
        tensor = word2vec_dict[target]
    return tensor
    
def search(target, modality, n=5):
    tensor = target_to_tensor(target, modality).detach().to("cuda")
    model_name = search_engine.valid_models(tensor, modality)[0]
    if modality == "image":
        #TODO: Make this less dumb
        tensor = tensor[None,:,:,:]
        
    embedding = search_engine.models[model_name].get_embedding(tensor)
    if modality == "image":
        #TODO: Make this less dumb
        embedding = embedding[0]
        
    results = []
    for index_key in search_engine.valid_indexes(embedding):
        dis, idx = search_engine.search(embedding, index_key, n = n)
        data = search_engine.data_from_idx(index_key, idx)
        dis = [float(d) for d in dis]
        results.append([index_key, list(dis), list(data)])
    return results

In [ ]:
search_engine.data_from_idx('nus-wide', [7,6])

In [ ]:
search_engine.data_from_idx('wiki_word2vec', [4343,6534])

In [ ]:
import random
import PIL

image = image_from_idx[random.randint(0,len(image_from_idx))]
image = "uploads/0004_159909288.jpg"
print(image)

In [ ]:
results = search(image, "image")
for r in results:
    print(r)

In [ ]:
results = search('moon', "text")

for r in results:
    for i in range(len(r[1])):
        print("Distance: ",r[1][i])
        if r[0][0] == "nus-wide":
            plt.imshow(PIL.Image.open(r[2][i]))
            plt.show()
        else:
            print(r[2][i])
    

In [ ]:
image = search_engine.data_from_idx('nus-wide', [30290])[0]
image = PIL.Image.open(image)
plt.imshow(image)